In [2]:
import altair as alt
import pandas as pd
import random
import json

from Modules.circuit import CircuitChart
from Modules import Run

In [3]:
with open('/Users/pau_matas/Desktop/DPA-visualization-tool/data/info.json') as f:
    info = json.load(f)

run = Run('/Users/pau_matas/Desktop/DPA-visualization-tool/data/TILK-E:420/420_Run0.csv', info['TILK-E:420'])
lapA = run.laps[0]
lapB = run.laps[1]

circuit = CircuitChart(seed = 420)
circuit.chart()

alt.Chart(...)

In [5]:
with open('/Users/pau_matas/Desktop/DPA-visualization-tool/data/TILK-E:420/turns.json') as f:
    info = json.load(f)
    
circuit.turns_chart(info)

alt.Chart(...)

In [17]:
from sklearn.neighbors import KDTree


lapA_kdtree = KDTree(lapA.df[['xPosition', 'yPosition']])
lapB_kdtree = KDTree(lapB.df[['xPosition', 'yPosition']])

delta = []
for i in range(100):
    door = circuit.middle_curve(circuit.middle_curve.t[-1] * (i+1)/100)
    Ai = lapA_kdtree.query([door])[1][0][0]
    Bi = lapB_kdtree.query([door])[1][0][0]
    delta.append(lapA.df['delta'][:Ai].sum() - lapB.df['delta'][:Bi].sum())

In [18]:
data = pd.DataFrame({
    'delta': delta,
    'bin': list(range(100)),
    'color': ['lapA' if d > 0 else 'lapB' for d in delta]
})

alt.Chart(data).mark_bar().encode(
    # alt x with a ratio from -1 to 1
    x=alt.X('delta:Q', scale=alt.Scale(domain=[-1, 1])),
    y=alt.Y('bin:N', axis=None),
    color=alt.condition(
        alt.datum.delta != 0,
        alt.Color('color:N', legend=None),
        alt.value('yellow')
    ),
    order='bin:Q',
    tooltip=['delta:Q']
)

alt.Chart(...)

In [5]:
# definim el primer valor de la llista
current_value = random.uniform(-0.8, 1)

# definim la llista buida
values = []

# iterem 1000 vegades per generar els valors de la llista
for i in range(300):
    # afegim el valor actual a la llista
    values.append(current_value)
    
    # calculem el valor següent amb un pas màxim de 0.05
    next_value = min(max(current_value + random.uniform(-0.04, 0.04), -1.2), 1)
    
    # actualitzem el valor actual per al següent pas del bucle
    current_value = next_value


In [6]:
# llegim el fitxer CSV amb pandas
data = pd.DataFrame([
    {'temps':i, 'distancia':values[i], 'sector': i//100} for i in range(300)]
)

display(data.head())

# definim la visualització de dades
chart = alt.Chart(data).mark_bar().encode(
    # alt x with a ratio from -1 to 1
    x=alt.X('distancia:Q', scale=alt.Scale(domain=[-1, 1])),
    y='temps:T',
    color=alt.condition(
        alt.datum.distancia > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    ),
    order='temps:Q',
    tooltip=['sector:N', 'distancia:Q']
)

sectors = pd.DataFrame([{'y': 0, 'label': 'Sector 1'}, {'y': 100, 'label': 'Sector 2'}, {'y': 200, 'label': 'Sector 3'}])
rule = alt.Chart(sectors).mark_rule(color='red').encode(
    y=alt.Y('y:T', axis=alt.Axis(title='temps')),
)
text = rule.mark_text(
    align='right',
    baseline='middle',
    dx=+200,
    dy=-5
).encode(
    text='label:N'
)
chart + rule + text


,temps,distancia,sector
0,0,0.756925,0
1,1,0.750191,0
2,2,0.726752,0
3,3,0.692964,0
4,4,0.692582,0


alt.LayerChart(...)